In [1]:
import pymysql

In [19]:
#저는 mysql root계정에 데이터베이스, 테이블을 만들었어요
conn = pymysql.connect(
    host='localhost',  # 호스트명
    user='root',  # 사용자명
    password='root1234',  # 비밀번호
    db='miniprojectdb',  # 데이터베이스명
    charset='utf8'  # 문자셋
)

In [20]:
cursor = conn.cursor((pymysql.cursors.DictCursor)) #데이터베이스와 소통을 위한 커서 생성 
#딕셔너리 타입으로 가져오기

In [21]:
sql = "SELECT * FROM predict1;" #여기서 todo는 테이블 이름. 실제 구현에서는 predect_오늘 날짜 가 테이블 이름이 될 것임
cursor.execute(sql) # 실행

2

In [22]:
result1 = cursor.fetchall() #커서의 실행 결과를 result에 담기

In [23]:
result1

[{'code': '565656',
  'name': '태현화학',
  'day1': 1,
  'day2': 15000,
  'day3': 10000,
  'day4': 123,
  'day5': 123123,
  'day6': 123213,
  'day7': 7,
  'day8': 8,
  'day9': 9,
  'day10': 10},
 {'code': '654321',
  'name': '태현자동차',
  'day1': 10000,
  'day2': 15000,
  'day3': 10000,
  'day4': 123,
  'day5': 123123,
  'day6': 123213,
  'day7': 7,
  'day8': 8,
  'day9': 9,
  'day10': 10}]

In [24]:
type(result1) #형태는 튜플이 아니라 리스트

list

In [31]:
#가져오는 것뿐 아니라 바로 붙일 수 있도록 준비하기
from datetime import datetime, timedelta
import holidays

In [48]:
kr_holidays = holidays.SouthKorea

In [58]:
kr_holidays = holidays.SouthKorea() #여기 괄호 안 붙였었음

def adjust_for_weekend_and_holiday(current_date, kr_holidays):
    while current_date.weekday() >= 5 or current_date in kr_holidays:
        current_date += timedelta(days=1)
    return current_date

result_list = [] #for문 이렇게 쓰는 거였지

current_company = None

for item in result1:
    company_name = item['name']

        # 회사가 변경되었을 때 새로운 리스트 생성
    if current_company != company_name:
        current_company = company_name
        result_list.append([])
    
    last_date = None #이걸 글로벌에 있다가 지역으로 들고옴.

    for i in range(1, 11): 
        date_key = f"day{i}"
        close_key = date_key

        if last_date is None:
            current_date = datetime.now() + timedelta(days=i)
            current_date = adjust_for_weekend_and_holiday(current_date, kr_holidays)

            date_value = current_date.strftime("%Y-%m-%d")
            close_value = item[date_key]
            
            entry = {'name': company_name, 'date': date_value, 'close': close_value}
            result_list[-1].append(entry)

            last_date = current_date

        # 이전에 찍힌 값에 +1
        else:

        # 현재 날짜에 i일을 더한 후 주말이나 휴일이면 조정. 이전에 찍힌 값에 +1을 하고, 주말이나 휴일이면 다음 평일로 조정합니다. 
            current_date = last_date + timedelta(days=1)
            current_date = adjust_for_weekend_and_holiday(current_date, kr_holidays)

            date_value = current_date.strftime("%Y-%m-%d")
            close_value = item[date_key]
            
            entry = {'name': company_name, 'date': date_value, 'close': close_value}
            result_list[-1].append(entry)

            last_date = current_date

print(result_list)

#휴일 제대로 반영되는지 체크. day를 6까지 늘리기. 일단 반영 안되고 근데 평일만 빼면 되나? 최소 공휴일은 포함된 정보를 가져와서 적용해야될 거 같은데
#휴일이면+1, 주말이면 다음 평일로 넘기는 로직은 26일만 3번 찍힘. 전에 찍힌 값에 +1을 해야 한다.
#회사별로 대괄호 부여하기
#성철형이랑 컬럼명 맞추기 이제 디테일로.
#TypeError: argument of type 'EntityLoader' is not iterable 반복가능한 객체가 아니래. if else가 for랑 같은 선에 있었음.

[[{'name': '태현화학', 'date': '2023-12-19', 'close': 1}, {'name': '태현화학', 'date': '2023-12-20', 'close': 15000}, {'name': '태현화학', 'date': '2023-12-21', 'close': 10000}, {'name': '태현화학', 'date': '2023-12-22', 'close': 123}, {'name': '태현화학', 'date': '2023-12-26', 'close': 123123}, {'name': '태현화학', 'date': '2023-12-27', 'close': 123213}, {'name': '태현화학', 'date': '2023-12-28', 'close': 7}, {'name': '태현화학', 'date': '2023-12-29', 'close': 8}, {'name': '태현화학', 'date': '2024-01-02', 'close': 9}, {'name': '태현화학', 'date': '2024-01-03', 'close': 10}], [{'name': '태현자동차', 'date': '2023-12-19', 'close': 10000}, {'name': '태현자동차', 'date': '2023-12-20', 'close': 15000}, {'name': '태현자동차', 'date': '2023-12-21', 'close': 10000}, {'name': '태현자동차', 'date': '2023-12-22', 'close': 123}, {'name': '태현자동차', 'date': '2023-12-26', 'close': 123123}, {'name': '태현자동차', 'date': '2023-12-27', 'close': 123213}, {'name': '태현자동차', 'date': '2023-12-28', 'close': 7}, {'name': '태현자동차', 'date': '2023-12-29', 'close': 8}, {'name'